In [ ]:
# train_dir = r'C:/Users/sk900/Downloads/Srija_Dissertation/Srija_Dissertation_2024/Brain_tumor_dataset/Training/'
# test_dir= r'C:/Users/sk900/Downloads/Srija_Dissertation/Srija_Dissertation_2024/Brain_tumor_dataset/Testing/'
train_dir = r'/content/drive/MyDrive/Srija_Dissertation_2024/Brain_tumor_dataset/Training/'
test_dir= r'/content/drive/MyDrive/Srija_Dissertation_2024/Brain_tumor_dataset/Testing/'
###Uncomment this last two codes if using google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import cv2
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import tensorly as tl
from tensorly.decomposition import tensor_train
from scipy.sparse.linalg import lsqr
import time
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
categories = ['pituitary', 'glioma', 'meningioma', 'notumor']

In [ ]:
img_height, img_width = 128, 128  # Resize images to this size


In [ ]:
# def load_images_from_folder(folder, categories, img_height, img_width, sample_size=None):
#     images = []
#     labels = []
#     for category in categories:
#         class_dir = os.path.join(folder, category)
#         if not os.path.isdir(class_dir):
#             continue
#         count = 0
#         for filename in os.listdir(class_dir):
#             if sample_size is not None and count >= sample_size:
#                 break
#             img_path = os.path.join(class_dir, filename)
#             img = cv2.imread(img_path)
#             if img is not None:
#                 img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
#                 img = cv2.resize(img, (img_height, img_width))
#                 img = img[:, :, np.newaxis]  # Add channel dimension
#                 images.append(img)
#                 labels.append(category)
#                 count += 1
#     return np.array(images), np.array(labels)
# sample_size = 1000
# X_train, y_train = load_images_from_folder(train_dir, categories, img_height, img_width, sample_size)
# X_test, y_test = load_images_from_folder(test_dir, categories, img_height, img_width, sample_size)


In [ ]:
def load_images_from_folder(folder, categories, img_height, img_width):
    images = []
    labels = []
    for category in categories:
        class_dir = os.path.join(folder, category)
        if not os.path.isdir(class_dir):
            continue
        for filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (img_height, img_width))
                img = img[:, :, np.newaxis]
                images.append(img)
                labels.append(category)
    return np.array(images), np.array(labels)

# Load training and testing data
X_train, y_train = load_images_from_folder(train_dir, categories, img_height, img_width)
X_test, y_test = load_images_from_folder(test_dir, categories, img_height, img_width)

In [ ]:
X_train.shape

(5882, 128, 128, 1)

In [ ]:
y_train.shape

(5882,)

In [ ]:
X_test.shape

(1321, 128, 128, 1)

In [ ]:
y_test.shape

(1321,)

In [ ]:
y_test

array(['pituitary', 'pituitary', 'pituitary', ..., 'notumor', 'notumor',
       'notumor'], dtype='<U10')

In [ ]:
# # Reshape and normalize the images
# X_train = X_train.reshape((X_train.shape[0], -1)) / 255.0
# X_test = X_test.reshape((X_test.shape[0], -1)) / 255.0

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
y_test

array([3, 3, 3, ..., 2, 2, 2])

In [ ]:
# X_train = X_train.mean(axis=-1)
# X_test = X_test.mean(axis=-1)


In [ ]:
X_train_tensor = X_train.reshape(-1, img_height, img_width, 1)
X_test_tensor = X_test.reshape(-1, img_height, img_width, 1)

In [ ]:
# # Normalize input data
# X_train_tensor = X_train_tensor / 255.0
# X_test_tensor = X_test_tensor / 255.0

In [ ]:
import numpy as np
from scipy.sparse.linalg import lsqr
from tensorly import tensor, unfold, fold
from tensorly.decomposition import tensor_train

In [ ]:
import numpy as np
from tensorly import tensor, unfold, fold, tensor_to_vec
from tensorly.decomposition import tensor_train
from tensorly.tt_tensor import tt_to_tensor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def tt_classifier_4d(X_train, y_train, X_test, ranks=[1, 30, 30, 30, 1]):
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")

    # Normalize the data
    scaler = StandardScaler()
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    X_train_normalized = scaler.fit_transform(X_train_flat).reshape(X_train.shape)
    X_test_normalized = scaler.transform(X_test_flat).reshape(X_test.shape)

    # Perform TT decomposition
    tt = tensor_train(X_train_normalized, rank=ranks)

    # Extract cores
    G1, G2, G3, G4 = tt.factors

    print(f"G1 shape: {G1.shape}")
    print(f"G2 shape: {G2.shape}")
    print(f"G3 shape: {G3.shape}")
    print(f"G4 shape: {G4.shape}")

    # Reconstruct the tensor
    X_train_reconstructed = tt_to_tensor(tt)
    print(f"Reconstructed X_train shape: {X_train_reconstructed.shape}")

    test_samples = X_test.shape[0]
    y_pred = np.zeros(test_samples, dtype=int)

    for i in range(test_samples):
        distances = np.sum((X_train_reconstructed - X_test_normalized[i])**2, axis=(1,2,3))
        y_pred[i] = y_train[np.argmin(distances)]

    return y_pred

# Use the classifier
y_pred = tt_classifier_4d(X_train, y_train, X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred) * 100

print(f"Accuracy: {accuracy:.2f}%")
print("y_test[:10]:", y_test[:10])
print("y_pred[:10]:", y_pred[:10])

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

X_train shape: (5882, 128, 128, 1)
y_train shape: (5882,)
X_test shape: (1321, 128, 128, 1)
G1 shape: (1, 5882, 30)
G2 shape: (30, 128, 30)
G3 shape: (30, 128, 1)
G4 shape: (1, 1, 1)
Reconstructed X_train shape: (5882, 128, 128, 1)
Accuracy: 97.27%
y_test[:10]: [3 3 3 3 3 3 3 3 3 3]
y_pred[:10]: [3 3 3 3 3 3 3 3 3 3]
Confusion Matrix:
[[277  23   0   0]
 [  6 304   4   2]
 [  0   1 404   0]
 [  0   0   0 300]]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')

precision_weighted = precision_score(y_test, y_pred, average='weighted')
recall_weighted = recall_score(y_test, y_pred, average='weighted')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-score: {f1_weighted:.4f}")

# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 97.2748
Macro Precision: 0.9723
Macro Recall: 0.9707
Macro F1-score: 0.9712
Weighted Precision: 0.9732
Weighted Recall: 0.9727
Weighted F1-score: 0.9727

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       300
           1       0.93      0.96      0.94       316
           2       0.99      1.00      0.99       405
           3       0.99      1.00      1.00       300

    accuracy                           0.97      1321
   macro avg       0.97      0.97      0.97      1321
weighted avg       0.97      0.97      0.97      1321



In [ ]:
import pandas as pd

In [ ]:
results_df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

results_df.head(1000)

,y_test,y_pred
0,3,3
1,3,3
2,3,3
3,3,3
4,3,3
...,...,...
995,2,2
996,2,2
997,2,2
998,2,2


In [ ]:
# def experiment_with_ranks(X_train, y_train, X_test, y_test):
#     rank_configs = [
#         [1, 10, 10, 10, 1],
#         [1, 20, 20, 20, 1],
#         [1, 30, 30, 30, 1],
#         [1, 40, 40, 40, 1],
#         [1, 50, 50, 50, 1],
#         [1, 55, 55, 55, 1],
#         [1, 60, 60, 60, 1],
#     ]

#     best_accuracy = 0
#     best_ranks = None

#     for ranks in rank_configs:
#         y_pred = tt_classifier_4d(X_train, y_train, X_test, ranks)
#         accuracy = accuracy_score(y_test, y_pred)
#         print(f"Ranks {ranks}: Accuracy {accuracy:.4f}")

#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_ranks = ranks

#     print(f"\nBest ranks: {best_ranks}")
#     print(f"Best accuracy: {best_accuracy:.4f}")

#     return best_ranks

# # Use this function to find the best ranks
# best_ranks = experiment_with_ranks(X_train, y_train, X_test, y_test)

In [ ]:
# def experiment_with_ranks(X_train, y_train, X_test, y_test):
#     rank_configs = [
#         [1, 2, 2, 1],
#         [1, 3, 3, 1],
#         [1, 5, 5, 1],
#         [1, 7, 7, 1],
#         [1, 10, 10, 1],
#         [1, 15, 15, 1],
#         [1, 20, 20, 1],
#         [1, 25, 25, 1],
#         [1, 30, 30, 1],
#     ]

#     best_accuracy = 0
#     best_ranks = None

#     for ranks in rank_configs:
#         y_pred = tt_classification(X_train, y_train, X_test, ranks)
#         accuracy = accuracy_score(y_test, y_pred)
#         print(f"Ranks {ranks}: Accuracy {accuracy:.4f}")

#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_ranks = ranks

#     print(f"\nBest ranks: {best_ranks}")
#     print(f"Best accuracy: {best_accuracy:.4f}")

#     return best_ranks

# # Use this function to find the best ranks
# best_ranks = experiment_with_ranks(X_train_tensor, y_train, X_test_tensor, y_test)